# Data Cleaning

In [25]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords

In [3]:
news = pd.read_csv('./data/news.csv', sep='~')
print('news.shape :', news.shape)
news.head(5)

news.shape : (66, 3)


,newspaper,text,date
0,bfm,"Parmi les trois foyers identifiés, l'un se tro...",NaN
1,bfm,"Entre le 1er mars et le 30 avril 2020, 11.328 ...",NaN
2,bfm,"Avant Novartis, c'est l'Organisation mondiale ...",NaN
3,bfm,À la suite de l'apparition de 27 cas en seulem...,NaN
4,bfm,Plusieurs cas de coronavirus ont été recensés ...,NaN


* **La stratégie**

La forme finale va être un ensemble de corpus : chaque journal aura son propre corpus qui sera la concatenation de leurs différents articles. La date de parution n'a pas d'importance. 
Pour "nettoyer" le texte, on va :
- tous mettre en miniscule, 
- remplacer les nombre par <\NB>\
- supprimer toute pontuation
- voire supprimer les stop words.

In [7]:
# Aggrégation des données
press = news.groupby('newspaper').agg({'text': pd.Series.tolist})
press['text'] = press.apply(lambda r: ' '.join(r['text']), axis=1)
press.head(5)

,text
newspaper,
20minutes,CORONAVIRUS Après une alerte lancée il y a tro...
bfm,"Parmi les trois foyers identifiés, l'un se tro..."
cnews,La France a enregistré 16 décès supplémentaire...
lacroix,Ne pas baisser la garde. C’est en résumé le me...
ledauphine,Un cas positif de Covid-19 a été détecté ce ve...


In [50]:
def clean_string(text):
    clean_text = text.lower()
    clean_text = re.sub('[.,!?#:/\"%€)@»«(➡️—-]', '', clean_text)
    clean_text = re.sub('’', ' ', clean_text)
    clean_text = re.sub('[éèëê]', 'e', clean_text)
    clean_text = re.sub('[àâ]', 'a', clean_text)
    clean_text = re.sub('[ïî]', 'i', clean_text)
    clean_text = re.sub('ù', 'u', clean_text)
    clean_text = re.sub('[öô]', 'o', clean_text)
    clean_text = re.sub('\w*\d\w*', '<NB>', clean_text)  # Remove numbers
    clean_text = re.sub("['\n\t]", ' ', clean_text)
    return clean_text

press['clean_text'] = press.apply(lambda r: clean_string(r['text']), axis=1)
press['clean_text'].iloc[0][:1000]

'coronavirus apres une alerte lancee il y a trois semaines sur un foyer de contaminations chez les saisonniers des bouchesdurhone les nouveaux cas diminuent  des travailleurs agricoles attendent pour se faire depister dans l ouest des bouchesdurhone  clement mahoudeau  afp  un foyer de contamination au <NB> chez les travailleurs saisonniers de l ouest des bouchesdurhone a ete detecte au debut du mois de juin  les services de l etat ont eu du mal pour placer en quarantaine ces travailleurs saisonniers a cause de leurs conditions d hebergements souvent compliquees  sur <NB> travailleurs testes positifs <NB> sont sortis de leur quarantaine  la bataille n est pas encore gagnee le prefet des bouchesdurhone pierre dartout et le directeur de l agence regionale de sante de provence alpes cote d azur philippe de mester ont fait le point sur le foyer de contaminations au <NB> chez des travailleurs saisonniers de l ouest du departement des bouchesdurhone  les nouvelles contaminations baissent dep

In [51]:
# Retirer les stops words
stop_words = set(stopwords.words('french'))
def remove_stop_words(text):
    return ' '.join([w for w in text.split(' ') if w not in stop_words])

press['relevent_text'] = press.apply(lambda r: remove_stop_words(r['clean_text']), axis=1)
press['relevent_text'].iloc[0][:1000]

'coronavirus apres alerte lancee a trois semaines foyer contaminations chez saisonniers bouchesdurhone nouveaux cas diminuent  travailleurs agricoles attendent faire depister ouest bouchesdurhone  clement mahoudeau  afp  foyer contamination <NB> chez travailleurs saisonniers ouest bouchesdurhone a ete detecte debut mois juin  services etat mal placer quarantaine travailleurs saisonniers a cause leurs conditions hebergements souvent compliquees  <NB> travailleurs testes positifs <NB> sortis quarantaine  bataille encore gagnee prefet bouchesdurhone pierre dartout directeur agence regionale sante provence alpes cote azur philippe mester fait point foyer contaminations <NB> chez travailleurs saisonniers ouest departement bouchesdurhone  nouvelles contaminations baissent depuis alerte hopital arles contamination <NB> trois travailleurs saisonniers ouest bouchesdurhone debut mois juin <NB> tests depistages ete realises <NB> cas positifs  parmi <NB> cas positifs <NB> passe cap quarantaine con

In [52]:
# Stem le text
from nltk.stem.snowball import FrenchStemmer

stemmer = FrenchStemmer()
press['stemmed_text'] = press.apply(lambda r: ' '.join([stemmer.stem(w) for w in r['relevent_text'].split(' ')]), axis=1)
press['stemmed_text'].iloc[0][:1000]

'coronavirus apre alert lance a trois semain foi contamin chez saisonni bouchesdurhon nouveau cas diminuent  travailleur agricol attendent fair depist ouest bouchesdurhon  clement mahoudeau  afp  foi contamin <nb> chez travailleur saisonni ouest bouchesdurhon a ete detect debut mois juin  servic etat mal plac quarantain travailleur saisonni a caus leur condit heberg souvent complique  <nb> travailleur test posit <nb> sort quarantain  bataill encor gagne prefet bouchesdurhon pierr dartout directeur agenc regional sant provenc alpe cot azur philipp mest fait point foi contamin <nb> chez travailleur saisonni ouest depart bouchesdurhon  nouvel contamin baissent depuis alert hopital arle contamin <nb> trois travailleur saisonni ouest bouchesdurhon debut mois juin <nb> test depistag ete realis <nb> cas posit  parm <nb> cas posit <nb> pass cap quarantain consider comm guer  a avanc philipp mest <nb> toujour plac quarantain  depart etion a taux positivit <nb> a <nb>  aujourd hui plutot autour 

In [53]:
# Save csv
press.to_csv('./data/press.csv', index=False)

## Document-Term matrix

In [54]:
press.index

Index(['20minutes', 'bfm', 'cnews', 'lacroix', 'ledauphine', 'lefigaro',
       'lemonde', 'lesechos', 'ouest-france'],
      dtype='object', name='newspaper')

In [55]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
dtm = cv.fit_transform(press['relevent_text'])

df_dtm = pd.DataFrame(dtm.toarray(), columns=cv.get_feature_names())
df_dtm = df_dtm.set_index(press.index)

df_dtm

,abattoir,abattoirs,abimer,abonde,abonnes,abord,aboutir,absence,absolue,absorber,...,youtube,yvelines,zero,zhongnan,zika,zone,zones,ça,œil,œuvre
newspaper,,,,,,,,,,,,,,,,,,,,,
20minutes,0,0,0,0,0,1,0,3,0,0,...,0,0,1,0,1,0,1,5,0,0
bfm,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,2,0,0
cnews,0,0,0,0,0,2,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
lacroix,0,0,0,0,0,1,0,0,0,1,...,0,0,2,0,0,0,0,0,2,2
ledauphine,0,0,1,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
lefigaro,0,0,0,0,0,1,1,4,0,0,...,1,0,0,0,0,1,0,1,0,0
lemonde,2,3,0,1,11,1,0,0,0,0,...,0,0,0,0,2,1,3,0,0,1
lesechos,1,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,2,1,0,0,0
ouest-france,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [ ]:
# Save matrix
df_dtm.to_pickle('df_dtm.pkl')
pickle.dump(cv, open("cv.pkl", "wb"))